In [22]:
publish.html("""
Cloud Formation:
<select id='action'>
 <option value="describe" selected>describe</option>
 <option value="create" >create</option>
 <option value="remove">remove</option>
 <option value="suspend">suspend</option>
 <option value="resume">resume</option>
</select>
<button id="run">Run</button>
""")
publish.javascript("""
$("#outputcontainer").remove()
$(".end_space").append("<div id='outputcontainer' class='container'><pre id='output'></pre></div>")
""")
publish.javascript("""
  window.now = function() { 
     var time = new Date(); 
     return time.getHours()+":"+time.getMinutes()+":"+time.getSeconds() 
  }
  $("#run").off("click")
  $("#run").on("click", function(data) {
  var data = "val action=\""+$("#action").val()+"\"\nval debug=false"
  IPython.notebook.get_cell(1).code_mirror.setValue(data) 
  IPython.notebook.select(1)
  IPython.notebook.execute_cells_below()
})
""")
def clean(s: String) = s.replaceAll("\n", """\\n""").replaceAll("'","""\\'""")
def message(s:String) = publish.javascript("$('#output').text(now()+'\\n"+clean(s)+"')")

Cloud Formation:
 
 describe 
 create 
 remove 
 suspend 
 resume 
 
 Run

defined function clean
defined function message

In [33]:
val action="create"
val debug=false

action: String = "create"
debug: Boolean = false

In [34]:
// options
import ammonite.ops._

val stackName = "test"
/// parameters
// template is for us-east-1
// change images for different regions
val region = "us-east-1"
val imageId = "ami-ae7bfdb8"
val instanceType = "t2.small"
val keyName = "atomic"
val template = pwd/'conf/"cloudformation.yml"

import ammonite.ops._


stackName: String = "test"
region: String = "us-east-1"
imageId: String = "ami-ae7bfdb8"
instanceType: String = "t2.small"
keyName: String = "atomic"
template: Path = /Users/msciab/Dropbox/Work/MacBookPro/MicheleSciabarra/Mosaico/cloud/conf/cloudformation.yml

In [35]:
import $ivy.`com.amazonaws:aws-java-sdk:1.11.22`

import $ivy.$                                   

In [36]:
import com.amazonaws.services.cloudformation.model._
import collection.JavaConverters._

val create  = action  == "create"
val remove  = action  == "remove"
val suspend = action  == "suspend"
val resume  = action  == "resume"
val describe = action  == "describe"

import com.amazonaws.services.cloudformation.model._

import collection.JavaConverters._


create: Boolean = true
remove: Boolean = false
suspend: Boolean = false
resume: Boolean = false
describe: Boolean = false

In [37]:
import com.amazonaws.services._
import com.amazonaws.services.cloudformation._

val cf = new AmazonCloudFormationClient() 
def status = {
    val stack = cf.describeStacks
        .getStacks.asScala
        .filter( _.getStackName == stackName).headOption
    stack.map(_.getStackStatus)
}
def statusLoop {
    val st = status
    message(st.getOrElse("-"))
    if(st.nonEmpty && st.get.endsWith("_IN_PROGRESS")) {
        Thread.sleep(1000)
        statusLoop
    }
}

import com.amazonaws.services._

import com.amazonaws.services.cloudformation._


cf: AmazonCloudFormationClient = com.amazonaws.services.cloudformation.AmazonCloudFormationClient@79762f8d
defined function status
defined function statusLoop

## Creation

In [38]:
// read template
import scala.io._
import com.amazonaws.services.cloudformation.model._

val cf = new cloudformation.AmazonCloudFormationClient

// create
def createRequest() = {
    import com.amazonaws.services.cloudformation._
    import com.amazonaws.services.cloudformation.model._
    val cfr = new CreateStackRequest()
    cfr.setStackName(stackName)
    val params =  Seq(
      new Parameter().withParameterKey("KeyName").withParameterValue(keyName),
      new Parameter().withParameterKey("InstanceType").withParameterValue(instanceType),
      new Parameter().withParameterKey("ImageId").withParameterValue(imageId)    
    ) 
    cfr.setParameters(params.asJava)
    cfr
}

if(create) {
    val cfr = createRequest()
    val body = Source.fromFile(template.toString).getLines.mkString("\n")
    cfr.setTemplateBody(body)
    // run request
    val res = cf.createStack(cfr)
    statusLoop
}

import scala.io._

import com.amazonaws.services.cloudformation.model._


cf: AmazonCloudFormationClient = com.amazonaws.services.cloudformation.AmazonCloudFormationClient@7ffffb8e
defined function createRequest

## Remove

In [39]:
// remove
def removeRequest() = 
    new DeleteStackRequest().withStackName(stackName)

if(remove) { 
    cf.deleteStack(removeRequest())
    statusLoop
}
//removeRequest()

defined function removeRequest

# Describe

In [40]:
import com.amazonaws.services.ec2._
import com.amazonaws.regions.{Region,Regions}

val ec2 = new AmazonEC2Client()
val regions = Regions.fromName(region)
ec2.setRegion(Region.getRegion(regions))

import com.amazonaws.services.ec2._

import com.amazonaws.regions.{Region,Regions}


ec2: AmazonEC2Client = com.amazonaws.services.ec2.AmazonEC2Client@7ccb391a
regions: Regions = US_EAST_1

In [41]:
import collection.JavaConverters._

val instances = for {
 reservation <- ec2.describeInstances.getReservations.asScala
 instance <- reservation.getInstances.asScala
} yield {
    instance
}

import collection.JavaConverters._


instances: collection.mutable.Buffer[Instance] = ArrayBuffer(
  {InstanceId: i-0365a60a28d0dd8bc,ImageId: ami-c58c1dd3,State: {Code: 48,Name: terminated},PrivateDnsName: ,PublicDnsName: ,StateTransitionReason: User initiated (2017-04-22 13:56:07 GMT),KeyName: atomic,AmiLaunchIndex: 0,ProductCodes: [],InstanceType: t2.small,LaunchTime: Sat Apr 22 14:37:20 CEST 2017,Placement: {AvailabilityZone: us-east-1d,GroupName: ,Tenancy: default,},Monitoring: {State: disabled},StateReason: {Code: Client.UserInitiatedShutdown,Message: Client.UserInitiatedShutdown: User initiated shutdown},Architecture: x86_64,RootDeviceType: ebs,RootDeviceName: /dev/xvda,BlockDeviceMappings: [],VirtualizationType: hvm,ClientToken: test-Master-TH2YZLQF0W8F,Tags: [{Key: aws:cloudformation:logical-id,Value: MasterInstance}, {Key: aws:cloudformation:stack-id,Value: arn:aws:cloudformation:us-east-1:794403675119:stack/test/4206b2b0-2758-11e7-891b-503acac41e99}, {Ke...

In [42]:
import com.amazonaws.services.ec2.model._
import collection.JavaConverters._

if(suspend) {
    val ids = instances.filter(_.getState.getName == "running").map(_.getInstanceId)
    ec2.stopInstances(new StopInstancesRequest(ids.asJava))
    message("SUSPENDING")
}

if(resume) {
     val stopped = instances.filter(_.getState.getName == "stopped").map(_.getInstanceId)
     ec2.startInstances(new StartInstancesRequest(stopped.asJava))
     message("RESUMING")
} 

def instName(inst: Instance) = inst.getTags.asScala.filter(_.getKey=="Name").map(_.getValue).headOption

if(describe) {
   val msg = instances.map { instance =>
         s"""id=${instance.getInstanceId} 
state=${instance.getState.getName} 
name=${instName(instance)} 
privateIp=${instance.getPrivateIpAddress}
publicIp=${instance.getPublicIpAddress}
----------"""
     }
    
    message(msg.mkString("\n"))
}

import com.amazonaws.services.ec2.model._

import collection.JavaConverters._


defined function instName